In [2]:
pip install requests


## Importing necessary libraries

In [21]:
import requests  
import zipfile
import io
import polars as pl
from glob import glob
import re


# Download and unzip yearly CMS data from 2017 - 2024

In [22]:
urls=[(year,
       f"https://data.cms.gov/provider-data/sites/default/files/archive/Hospitals/{year}/hospitals{'_archive' if year <=2020 else ''}_{year}.zip")
      for year in range (2017,2025)]
urls


[(2017,
  'https://data.cms.gov/provider-data/sites/default/files/archive/Hospitals/2017/hospitals_archive_2017.zip'),
 (2018,
  'https://data.cms.gov/provider-data/sites/default/files/archive/Hospitals/2018/hospitals_archive_2018.zip'),
 (2019,
  'https://data.cms.gov/provider-data/sites/default/files/archive/Hospitals/2019/hospitals_archive_2019.zip'),
 (2020,
  'https://data.cms.gov/provider-data/sites/default/files/archive/Hospitals/2020/hospitals_archive_2020.zip'),
 (2021,
  'https://data.cms.gov/provider-data/sites/default/files/archive/Hospitals/2021/hospitals_2021.zip'),
 (2022,
  'https://data.cms.gov/provider-data/sites/default/files/archive/Hospitals/2022/hospitals_2022.zip'),
 (2023,
  'https://data.cms.gov/provider-data/sites/default/files/archive/Hospitals/2023/hospitals_2023.zip'),
 (2024,
  'https://data.cms.gov/provider-data/sites/default/files/archive/Hospitals/2024/hospitals_2024.zip')]

In [23]:
for year, url in urls:
    try:
        r = requests.get(url)  # Corrected from 'requests-get' to 'requests.get'
        assert r.ok
        z = zipfile.ZipFile(io.BytesIO(r.content))
        z.extractall("./data")
        print(f"Successfully downloaded and unzipped {year}\n\n")
    except Exception as e:
        print(f"Problem with the zip file for {year}: {e}\n\n")

Successfully downloaded and unzipped 2017


Successfully downloaded and unzipped 2018


Successfully downloaded and unzipped 2019


Successfully downloaded and unzipped 2020


Successfully downloaded and unzipped 2021


Successfully downloaded and unzipped 2022


Successfully downloaded and unzipped 2023


Successfully downloaded and unzipped 2024




# Unzip monthly files - Attempt 1

In [24]:
%%bash
ls -al data

total 546044
drwxrwxrwx 1 root root     4096 Oct 17 07:46 .
drwxrwxrwx 1 root root     4096 Oct 17 07:46 ..
-rwxrwxrwx 1 root root 51900380 Oct 17 07:46 hos_archive_04_2017.zip
-rwxrwxrwx 1 root root 16666309 Oct 17 07:46 hos_revised_flatfiles_archive_01_2018.zip
-rwxrwxrwx 1 root root 16188156 Oct 17 07:46 hos_revised_flatfiles_archive_01_2020.zip
-rwxrwxrwx 1 root root 18544644 Oct 17 07:46 hos_revised_flatfiles_archive_03_2019.zip
-rwxrwxrwx 1 root root 17177735 Oct 17 07:46 hos_revised_flatfiles_archive_04_2017.zip
-rwxrwxrwx 1 root root 17719823 Oct 17 07:46 hos_revised_flatfiles_archive_04_2019.zip
-rwxrwxrwx 1 root root 16306683 Oct 17 07:46 hos_revised_flatfiles_archive_04_2020.zip
-rwxrwxrwx 1 root root 16097035 Oct 17 07:46 hos_revised_flatfiles_archive_05_2018.zip
-rwxrwxrwx 1 root root 16941650 Oct 17 07:46 hos_revised_flatfiles_archive_07_2017.zip
-rwxrwxrwx 1 root root 16203900 Oct 17 07:46 hos_revised_flatfiles_archive_07_2018.zip
-rwxrwxrwx 1 root root 21768646 Oct 17 0

In [31]:
(all_monthly_zip :=
 glob('./data/*zip')
)

['./data\\hospitals_01_2021.zip',
 './data\\hospitals_01_2022.zip',
 './data\\hospitals_01_2023.zip',
 './data\\hospitals_01_2024.zip',
 './data\\hospitals_03_2021.zip',
 './data\\hospitals_04_2021.zip',
 './data\\hospitals_04_2022.zip',
 './data\\hospitals_04_2023.zip',
 './data\\hospitals_04_2024.zip',
 './data\\hospitals_07_2021.zip',
 './data\\hospitals_07_2022.zip',
 './data\\hospitals_07_2023.zip',
 './data\\hospitals_07_2024.zip',
 './data\\hospitals_10_2021.zip',
 './data\\hospitals_10_2022.zip',
 './data\\hospitals_10_2023.zip',
 './data\\hospitals_11_2023.zip',
 './data\\hospitals_archive_07_2020.zip',
 './data\\hospitals_archive_10_2020.zip',
 './data\\hos_archive_04_2017.zip',
 './data\\hos_revised_flatfiles_archive_01_2018.zip',
 './data\\hos_revised_flatfiles_archive_01_2020.zip',
 './data\\hos_revised_flatfiles_archive_03_2019.zip',
 './data\\hos_revised_flatfiles_archive_04_2017.zip',
 './data\\hos_revised_flatfiles_archive_04_2019.zip',
 './data\\hos_revised_flatfiles_

In [42]:
for path in all_monthly_zip:
    with zipfile.ZipFile(path, 'r') as zip_obj:
        zip_obj.extractall('./data')
        print(f'Successfully unzipped file={path}')

Successfully unzipped file=./data\hospitals_01_2021.zip
Successfully unzipped file=./data\hospitals_01_2022.zip
Successfully unzipped file=./data\hospitals_01_2023.zip
Successfully unzipped file=./data\hospitals_01_2024.zip
Successfully unzipped file=./data\hospitals_03_2021.zip
Successfully unzipped file=./data\hospitals_04_2021.zip
Successfully unzipped file=./data\hospitals_04_2022.zip
Successfully unzipped file=./data\hospitals_04_2023.zip
Successfully unzipped file=./data\hospitals_04_2024.zip
Successfully unzipped file=./data\hospitals_07_2021.zip
Successfully unzipped file=./data\hospitals_07_2022.zip
Successfully unzipped file=./data\hospitals_07_2023.zip
Successfully unzipped file=./data\hospitals_07_2024.zip
Successfully unzipped file=./data\hospitals_10_2021.zip
Successfully unzipped file=./data\hospitals_10_2022.zip
Successfully unzipped file=./data\hospitals_10_2023.zip
Successfully unzipped file=./data\hospitals_11_2023.zip
Successfully unzipped file=./data\hospitals_arch

# Unzip monthly files - Attempt 2

In [43]:
m_and_y = re.compile(r'^\./data\\hos[a-z_]*(\d{2})_(\d{4})\.zip$')

(folder_and_path :=
    [(f'./data/hospitals_{m_and_y.match(p).group(2)}_{m_and_y.match(p).group(1)}', # creating folders saving data into
      p, # path to the folder
     )
     for p in all_monthly_zip]
)

#         folder              -              path

[('./data/hospitals_2021_01', './data\\hospitals_01_2021.zip'),
 ('./data/hospitals_2022_01', './data\\hospitals_01_2022.zip'),
 ('./data/hospitals_2023_01', './data\\hospitals_01_2023.zip'),
 ('./data/hospitals_2024_01', './data\\hospitals_01_2024.zip'),
 ('./data/hospitals_2021_03', './data\\hospitals_03_2021.zip'),
 ('./data/hospitals_2021_04', './data\\hospitals_04_2021.zip'),
 ('./data/hospitals_2022_04', './data\\hospitals_04_2022.zip'),
 ('./data/hospitals_2023_04', './data\\hospitals_04_2023.zip'),
 ('./data/hospitals_2024_04', './data\\hospitals_04_2024.zip'),
 ('./data/hospitals_2021_07', './data\\hospitals_07_2021.zip'),
 ('./data/hospitals_2022_07', './data\\hospitals_07_2022.zip'),
 ('./data/hospitals_2023_07', './data\\hospitals_07_2023.zip'),
 ('./data/hospitals_2024_07', './data\\hospitals_07_2024.zip'),
 ('./data/hospitals_2021_10', './data\\hospitals_10_2021.zip'),
 ('./data/hospitals_2022_10', './data\\hospitals_10_2022.zip'),
 ('./data/hospitals_2023_10', './data\\h

In [44]:
for folder, path in folder_and_path:
    with zipfile.ZipFile(path, 'r') as zip_obj:       # unzipping the folders 
        zip_obj.extractall(folder)                    # saving the files into the folder
        print(f'Successfully unzipped file={path}')

Successfully unzipped file=./data\hospitals_01_2021.zip
Successfully unzipped file=./data\hospitals_01_2022.zip
Successfully unzipped file=./data\hospitals_01_2023.zip
Successfully unzipped file=./data\hospitals_01_2024.zip
Successfully unzipped file=./data\hospitals_03_2021.zip
Successfully unzipped file=./data\hospitals_04_2021.zip
Successfully unzipped file=./data\hospitals_04_2022.zip
Successfully unzipped file=./data\hospitals_04_2023.zip
Successfully unzipped file=./data\hospitals_04_2024.zip
Successfully unzipped file=./data\hospitals_07_2021.zip
Successfully unzipped file=./data\hospitals_07_2022.zip
Successfully unzipped file=./data\hospitals_07_2023.zip
Successfully unzipped file=./data\hospitals_07_2024.zip
Successfully unzipped file=./data\hospitals_10_2021.zip
Successfully unzipped file=./data\hospitals_10_2022.zip
Successfully unzipped file=./data\hospitals_10_2023.zip
Successfully unzipped file=./data\hospitals_11_2023.zip
Successfully unzipped file=./data\hospitals_arch

## Unzip monthly files - Attempt 3 - Unzip into monthly folder - adjusting for 2022 and 2023

In [45]:
m_and_y = re.compile(r'^\./data\\hos[a-z_]*(\d{2})_(\d{4})\.zip$')     # regx to pull off 'month' and 'year'

# helper functions to extract corresponding pieces
# use 'match' and 'group' 
month    = lambda p: m_and_y.match(p).group(1)
year     = lambda p: m_and_y.match(p).group(2)
is_22_23 = lambda p: year(p) in ['2022', '2023']     # return True if year is either 2022 or 2023

folder = lambda p: './data' if is_22_23(p) else f'./data/hospitals_{month(p)}_{year(p)}'

(folder_and_path := 
    [(folder(p),
     p,
     )
     for p in all_monthly_zip]
)

[('./data/hospitals_01_2021', './data\\hospitals_01_2021.zip'),
 ('./data', './data\\hospitals_01_2022.zip'),
 ('./data', './data\\hospitals_01_2023.zip'),
 ('./data/hospitals_01_2024', './data\\hospitals_01_2024.zip'),
 ('./data/hospitals_03_2021', './data\\hospitals_03_2021.zip'),
 ('./data/hospitals_04_2021', './data\\hospitals_04_2021.zip'),
 ('./data', './data\\hospitals_04_2022.zip'),
 ('./data', './data\\hospitals_04_2023.zip'),
 ('./data/hospitals_04_2024', './data\\hospitals_04_2024.zip'),
 ('./data/hospitals_07_2021', './data\\hospitals_07_2021.zip'),
 ('./data', './data\\hospitals_07_2022.zip'),
 ('./data', './data\\hospitals_07_2023.zip'),
 ('./data/hospitals_07_2024', './data\\hospitals_07_2024.zip'),
 ('./data/hospitals_10_2021', './data\\hospitals_10_2021.zip'),
 ('./data', './data\\hospitals_10_2022.zip'),
 ('./data', './data\\hospitals_10_2023.zip'),
 ('./data', './data\\hospitals_11_2023.zip'),
 ('./data/hospitals_07_2020', './data\\hospitals_archive_07_2020.zip'),
 (